# Classical Supervised Learning

Supervised machine learning is when we use a dataset **with known structure and/or features** to teach an algorithm how to approximate a function

$$ y = f(x) $$

between the data $x$ and the known outcome $y$. Classical machine learning is the machine learning algorithms that are not deep neural networks. Thus classical supervised learning is achieved by approximating the function $f$ using alternative methods to deep neural networks.

In this exercise notebook we will look at different datasets, their structure and how to classify the data in the datasets. On top of this we will see how shallow neural networks can be used as **universal function approximators**.

We will start by classifying the same dataset using different classification methods and how this affects the accuracy and the placement of the decision boundary in the dataset.

In [ ]:
%matplotlib notebook
%config InlineBackend.figure_format = "retina"

import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import numpy as np
from matplotlib.colors import ListedColormap
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
import sklearn.gaussian_process.kernels as kernels
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from scipy import special
from IPython.core.display import display,HTML
display(HTML("<style>.container{ width: 100% !important;} <\style>"))

## Aside: plotting code

I have provided a plotting code to plot the results of a classifier and the decision boundaries. This is provided as an easy to use function as I didn't see the need for anyone else to have to struggle with `matplotlib` to make these plots.

In [ ]:
def class_plot(data,targets,feature_names,classifier,rang=0.25):
    '''
    A function to make the boundary contour plots.
    
    Parameters
    ----------
    data : numpy.ndarray
        The feature data.
    targets : numpy.ndarray
        The labels of the data samples.
    feature_names : list
        A list of the names of the features to be compared.
    classifier : various
        The class instance for the classifier being used after training.
    rang : float
        Defines the range of the x-axis and y-axis.
    '''
    
    x_min, x_max = data[:,0].min() - rang, data[:,0].max() + rang
    y_min, y_max = data[:,1].min() - rang, data[:,1].max() + rang
    
    xx, yy = np.meshgrid(np.arange(x_min,x_max,0.02),np.arange(y_min,y_max,0.02))
    
    if hasattr(classifier,"decision_function"):
        Z = classifier.decision_function(np.c_[xx.ravel(),yy.ravel()]).reshape(xx.shape)
    else:
        Z = classifier.predict_proba(np.c_[xx.ravel(),yy.ravel()])[:,1].reshape(xx.shape)

    cm = ListedColormap(["#FF0000","#0000FF"])

    plt.figure()
    plt.contourf(xx,yy,Z,cmap="RdBu",alpha=0.8)
    plt.scatter(data[:,0],data[:,1],c=targets,cmap=cm)
    plt.ylabel(feature_names[1])
    plt.xlabel(feature_names[0])

## Classification Dataset

Next we must load the dataset. The data is saved in ```.npz``` format. This is a data format native to ```numpy``` and works to save multiple ```numpy``` arrays to disk without losing any information. For more information check out the [`scipy` documentation](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.lib.format.html#module-numpy.lib.format). ```.npz``` is essentially a ```.zip``` of ```numpy``` arrays. Loading a ```.npz``` file returns a ```dict```-like object where the arrays have the keys corresponding to the names assigned when saving the file. e.g.

```python
>>> import numpy as np
>>> a = np.array([[1,2],[3,4]])
>>> np.savez_compressed("a.npz",data=a)
>>> f = np.load("a.npz")
>>> f["data"] == a
array([[True, True],
       [True, True]])
```

In the example above, we use the function ```savez_compressed``` to save the data which creates a compressed ```.npz``` file but the function ```savez``` to create an uncompressed ```.npz``` file also exists (but when loading the data will make your code slower. ```.npz``` files preserve the structure of the arrays so there is no need to worry about doing any data manipulations when working with these files.

In [ ]:
data = np.load("data.npz")

In [ ]:
data.keys() #this tells you what is in the dataset

## Classification Method #1: The Perceptron

The perceptron is the simplest machine learning algorithm as it consists of a linear transformation followed by a non-linear step function to calculate its output. This is essentially one "neuron" of a layer of a neural network. This can be trained for only linear tasks and our dataset is a non-linear binary classification set which can be reflected in the perfomance.

1. Create a perceptron instance.
2. Next, fit this to the data and check the accuracy.
3. Lastly, plot your reduced data showing the decision boundaries using the plotting code provided.

**Feel free to change the tolerance level in the perceptron class instance to try to improve convergence (or make it worse).**
**Make sure to always choose** `random_state` **to be the same number when creating the class instance otherwise it will train differently every time.**

## Classification Method #2: K-Nearest Neighbours

K-nearest neighbours is a instance-based classifier and rather than learning a general model, this method learns information about the training data (i.e. where they lie in the 2D plane) to choose where to put the data it has not seen before. Classification in this case is performed by a majority vote, the distance from a point to the k nearest neighbours is calculated and the minimal distance defines what class the point belongs to.

The process is the same as that above.

**Try different numbers of nearest neighbours to see which gives you the best accuracy.**

## Classification Method #3: Gaussian Processes

A Gaussian process is one in which each of the classes in the dataset is assumed to be normally distributed i.e. drawn from a multivariate normal distribution. The distribution of the entire process is then the joint distribution of the features and is thus a continuous distribution in the space we are working.

We start from the assumption that each class is drawn from a normal with zero mean and a covariance matrix that we would like to learn to represent the data. This covariance matrix is often referred to as the kernel of the Gaussian process and is something that can be changed to yield different results. Starting from a Gaussian and calculating the covariance between the points in the classes allows the distribution of each class to be jointly updated until convergence.

**For the dataset provided, try different covariance matrices to see which converges the best.**

## Classification Method #4: Support Vector Machine (SVM)

The last classification technique we will employ is the support vector machine (SVM). Support vector machines find a non-linear transformation in the dimension one higher than the dimension of the data to find a hyperplane which makes the data linearly separable. The "support vectors" are the points in the plane closest to the boundary of this hyperplane to help refine the position of it.

**Try different kernels to see which transformation yields the best solution.**

## Function Approximation using SVMs and Neural Networks

The follow is an exercise on function approximating with support vector machines (SVMs) and neural networks. The neural network here has one layer between the input and output and is well-versed at function approximation. The dataset we use here is `voigt.npz`. This is the Voigt H function for damping factor a = 1, i.e.,

$$ H (a, v) = \frac{a}{\pi} \int_{0}^{\infty} \frac{e^{- (v^{\prime})^{2}}}{(a^{2} + (v - v^{\prime})^{2}} dv^{\prime} $$

The profile made for a range in v and a = 1 then has random Gaussian noise applied to it and we want to see how well we can approximate this, first using an SVM and second using a shallow neural network. After learning the function try plotting the true line profile (also given in the data file) to see how well your fit done.

In [ ]:
voigt = np.load("voigt.npz")

In [ ]:
voigt.keys()

For the neural network approach we must define the model, the datasets, the loss function and the numerical method for optimisation. A template for this example is shown below but can be edited if you feel the need to.

In [ ]:
model = nn.Sequential(
    nn.Linear(1,100),
    nn.Sigmoid(),
    nn.Linear(100,1)
)

loss_fn = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(),lr=0.01)

In [ ]:
xy = np.stack((X,y),axis=1)
xy_train, xy_test = train_test_split(xy,test_size=0.1)
x_train, y_train = xy_train[:,0], xy_train[:,1]
x_test, y_test = xy_test[:,0], xy_test[:,1]

In [ ]:
train_loader = DataLoader(TensorDataset(torch.from_numpy(x_train).float(),torch.from_numpy(y_train).float()),shuffle=True)
test_loader = DataLoader(TensorDataset(torch.from_numpy(x_test).float(),torch.from_numpy(y_test).float()))

## More complex function approximation using NNs

We now generalise our neural network to learn the Voigt profile for arbitrary damping factor, a (where previously we have taken a = 1). Now load in the data containing many Voigt functions. Here we train a shallow neural network to try to learn the general voigt function (the data is already provided in `general_voigt.npz`).

Try training for 500 epochs and see how well you can generate a Voigt profile by creating your own with damping factor 0 $\leq$ a $\leq$ 2. The function for making a Voigt profile is provided below.

In [ ]:
def voigt_H(a,v):
    z = v + 1j * a
    return special.wofz(z).real

In [ ]:
general_voigt = np.load("general_voigt.npz")

In [ ]:
general_voigt.keys()